In [3]:
# Install the core library for Transformers
!pip install transformers

# Install PyTorch or TensorFlow (the backend framework)
!pip install torch

## BERT Sentiment Analysis 
### Bottom 20 Outlets

In [18]:
from pathlib import Path
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# -----------------------------
# Configuration
# -----------------------------
BASE_DIR = Path.cwd().parent 
REVIEWS_DIR = BASE_DIR / "Reviews" / "All"
OVERVIEW_DIR = BASE_DIR / "Reviews" / "Overview"

MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"
TEXT_COLUMN = "text"
BATCH_SIZE = 32

# Existing VADER summary for bottom 20
VADER_FILE = OVERVIEW_DIR / "bottom_20_outlets_with_sentiment.csv"

# Output CSV
OUTPUT_FILE = OVERVIEW_DIR / "bottom_20_outlets_with_BERT_sentiment.csv"

# -----------------------------
# Step 1: Load bottom 20 outlets from VADER
# -----------------------------
vader_df = pd.read_csv(VADER_FILE)
bottom20_outlets = vader_df['outlet'].tolist()
bottom20_normalized = [x.strip().lower() for x in bottom20_outlets]
print(f"📊 Loaded {len(bottom20_outlets)} bottom 20 outlets from VADER summary")

# -----------------------------
# Step 2: Collect all review CSVs
# -----------------------------
csv_files = list(REVIEWS_DIR.glob("*.csv"))
if not csv_files:
    raise FileNotFoundError(f"❌ No CSV files found in {REVIEWS_DIR}")

print(f"📂 Found {len(csv_files)} outlet CSV files in '{REVIEWS_DIR}'")

# -----------------------------
# Step 3: Load BERT sentiment model
# -----------------------------
print(f"⚙️ Loading BERT model: {MODEL_NAME} ...")
sentiment_pipeline = pipeline("sentiment-analysis", model=MODEL_NAME, tokenizer=MODEL_NAME)

# -----------------------------
# Step 4: Process each outlet
# -----------------------------
bert_summaries = []

MAX_LENGTH = 512  # max tokens for Roberta models

for csv_path in tqdm(csv_files, desc="Processing outlets"):
    outlet_name = csv_path.stem
    outlet_name_clean = outlet_name.replace("_reviews", "").strip().lower()

    if outlet_name_clean not in bottom20_normalized:
        continue

    try:
        df = pd.read_csv(csv_path)
        if TEXT_COLUMN not in df.columns:
            continue
        df = df.dropna(subset=[TEXT_COLUMN])
        if df.empty:
            continue

        texts = df[TEXT_COLUMN].astype(str).tolist()

        # Truncate texts to MAX_LENGTH
        texts_truncated = [t[:MAX_LENGTH*4] for t in texts]  # ~4 chars per token

        # Run BERT sentiment
        results = sentiment_pipeline(texts_truncated, batch_size=BATCH_SIZE)
        df['bert_label'] = [r['label'] for r in results]
        df['bert_score'] = [r['score'] for r in results]

        # Normalize labels
        label_map = {
            'LABEL_0': 'negative',
            'LABEL_1': 'neutral',
            'LABEL_2': 'positive',
            'NEGATIVE': 'negative',
            'NEUTRAL': 'neutral',
            'POSITIVE': 'positive'
        }
        df['bert_cat'] = df['bert_label'].map(label_map).fillna(df['bert_label'])

        # Compute counts and percentages
        total_reviews = len(df)
        pos = df['bert_cat'].eq('positive').sum()
        neu = df['bert_cat'].eq('neutral').sum()
        neg = df['bert_cat'].eq('negative').sum()

        bert_summary = {
            "outlet_name": outlet_name_clean,
            "total_reviews_bert": total_reviews,
            "positive_bert": int(pos),
            "neutral_bert": int(neu),
            "negative_bert": int(neg),
            "pct_positive_bert": round(pos / total_reviews * 100, 2),
            "pct_neutral_bert": round(neu / total_reviews * 100, 2),
            "pct_negative_bert": round(neg / total_reviews * 100, 2)
        }
        bert_summaries.append(bert_summary)

    except Exception as e:
        print(f"❌ Error processing {outlet_name}: {e}")


# -----------------------------
# Step 5: Merge with VADER summary
# -----------------------------
bert_df = pd.DataFrame(bert_summaries)
if bert_df.empty:
    raise ValueError("❌ No BERT results were generated.")

# Ensure consistent column for merge
vader_df['outlet_clean'] = vader_df['outlet'].str.strip().str.lower()
merged_df = pd.merge(
    vader_df,
    bert_df,
    left_on='outlet_clean',
    right_on='outlet_name',
    how='left'
).drop(columns=['outlet_clean', 'outlet_name'])

# -----------------------------
# Step 6: Save final CSV
# -----------------------------
merged_df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Saved BERT-augmented bottom 20 outlets CSV: {OUTPUT_FILE}")

# Preview
print("\nSample rows:")
print(merged_df.head())




📊 Loaded 20 bottom 20 outlets from VADER summary
📂 Found 134 outlet CSV files in '/Users/breann/Documents/GitHub/IS434-Anytime-Fitness/Google-Reviews/Reviews/All'
⚙️ Loading BERT model: cardiffnlp/twitter-roberta-base-sentiment ...


Device set to use cpu
Processing outlets: 100%|██████████| 134/134 [02:48<00:00,  1.26s/it]

✅ Saved BERT-augmented bottom 20 outlets CSV: /Users/breann/Documents/GitHub/IS434-Anytime-Fitness/Google-Reviews/Reviews/Overview/bottom_20_outlets_with_BERT_sentiment.csv

Sample rows:
                                outlet  avg_rating  total_reviews  positive  \
0                      Anytime Fitness        4.16            622       291   
1                  Anytime Fitness NEX        3.06            295        88   
2        Anytime Fitness Tanjong Pagar        4.33            267       201   
3  Anytime Fitness Jurong East Central        3.90            249       149   
4      Anytime Fitness Northpoint City        3.00            244        84   

   neutral  negative  no_text  pct_positive  pct_neutral  pct_negative  \
0      300        31      183         46.78        48.23          4.98   
1      147        60      124         29.83        49.83         20.34   
2       46        20       42         75.28        17.23          7.49   
3       61        39       45         59.8

In [20]:

# Use Path and repo_root if you computed it earlier in the notebook
from pathlib import Path
import pandas as pd

repo_root = Path.cwd().parent
csv_path = (repo_root / "Reviews" / "Overview" / "bottom_20_outlets_with_BERT_sentiment.csv").resolve()
df = pd.read_csv(csv_path)
df.head()


# df = bottom_20_with_sentiment.csv already loaded
df['total_vader'] = df['positive'] + df['neutral'] + df['negative']
df['total_bert'] = df['positive_bert'] + df['neutral_bert'] + df['negative_bert']

# Compute agreement in percentages
df['pct_positive_diff'] = abs(df['pct_positive'] - df['pct_positive_bert'])
df['pct_neutral_diff'] = abs(df['pct_neutral'] - df['pct_neutral_bert'])
df['pct_negative_diff'] = abs(df['pct_negative'] - df['pct_negative_bert'])

# Optional: a single measure of disagreement
df['avg_pct_diff'] = df[['pct_positive_diff','pct_neutral_diff','pct_negative_diff']].mean(axis=1)

# Sort by disagreement to see which outlets differ most
df.sort_values('avg_pct_diff', ascending=False)[['outlet','pct_positive','pct_positive_bert','pct_neutral','pct_neutral_bert','pct_negative','pct_negative_bert','avg_pct_diff']]



,outlet,pct_positive,pct_positive_bert,pct_neutral,pct_neutral_bert,pct_negative,pct_negative_bert,avg_pct_diff
15,Anytime Fitness Tiong Bahru Plaza,33.88,63.79,58.68,13.79,7.44,22.41,29.923333
6,Anytime Fitness Bedok Central,48.61,77.42,46.30,5.65,5.09,16.94,27.103333
1,Anytime Fitness NEX,29.83,28.65,49.83,15.20,20.34,56.14,23.870000
4,Anytime Fitness Northpoint City,34.43,28.93,39.75,10.06,25.82,61.01,23.460000
18,Anytime Fitness Kovan,35.51,51.52,43.93,9.09,20.56,39.39,23.226667
16,Anytime Fitness West Coast Plaza,40.83,62.50,49.17,16.67,10.00,20.83,21.666667
12,Anytime Fitness Paya Lebar,36.96,42.67,50.72,18.67,12.32,38.67,21.370000
10,Anytime Fitness Upper Cross Street,40.85,50.56,42.25,11.24,16.90,38.20,20.673333
14,Anytime Fitness Jalan Besar,47.33,64.56,45.80,15.19,6.87,20.25,20.406667
5,Anytime Fitness ACE The Place CC,55.79,78.48,36.48,6.96,7.73,14.56,19.680000


## BERT Analysis 
### Top 20 Outlets

In [23]:
from pathlib import Path
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# -----------------------------
# Configuration
# -----------------------------
BASE_DIR = Path.cwd().parent 
REVIEWS_DIR = BASE_DIR / "Reviews" / "All"
OVERVIEW_DIR = BASE_DIR / "Reviews" / "Overview"

MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"
TEXT_COLUMN = "text"
BATCH_SIZE = 32

# Existing VADER summary for top 20
VADER_FILE = OVERVIEW_DIR / "top_20_outlets_with_sentiment.csv"

# Output CSV
OUTPUT_FILE = OVERVIEW_DIR / "top_20_outlets_with_BERT_sentiment.csv"

# -----------------------------
# Step 1: Load top 20 outlets from VADER
# -----------------------------
vader_df = pd.read_csv(VADER_FILE)
top20_outlets = vader_df['outlet'].tolist()
top20_normalized = [x.strip().lower() for x in top20_outlets]
print(f"📊 Loaded {len(top20_outlets)} top 20 outlets from VADER summary")

# -----------------------------
# Step 2: Collect all review CSVs
# -----------------------------
csv_files = list(REVIEWS_DIR.glob("*.csv"))
if not csv_files:
    raise FileNotFoundError(f"❌ No CSV files found in {REVIEWS_DIR}")

print(f"📂 Found {len(csv_files)} outlet CSV files in '{REVIEWS_DIR}'")

# -----------------------------
# Step 3: Load BERT sentiment model
# -----------------------------
print(f"⚙️ Loading BERT model: {MODEL_NAME} ...")
sentiment_pipeline = pipeline("sentiment-analysis", model=MODEL_NAME, tokenizer=MODEL_NAME)

# -----------------------------
# Step 4: Process each outlet
# -----------------------------
bert_summaries = []

MAX_LENGTH = 512  # max tokens for Roberta models

for csv_path in tqdm(csv_files, desc="Processing outlets"):
    outlet_name = csv_path.stem
    outlet_name_clean = outlet_name.replace("_reviews", "").strip().lower()

    if outlet_name_clean not in top20_normalized:
        continue

    try:
        df = pd.read_csv(csv_path)
        if TEXT_COLUMN not in df.columns:
            continue
        df = df.dropna(subset=[TEXT_COLUMN])
        if df.empty:
            continue

        texts = df[TEXT_COLUMN].astype(str).tolist()

        # Truncate texts to MAX_LENGTH
        texts_truncated = [t[:MAX_LENGTH*4] for t in texts]  # ~4 chars per token

        # Run BERT sentiment
        results = sentiment_pipeline(texts_truncated, batch_size=BATCH_SIZE)
        df['bert_label'] = [r['label'] for r in results]
        df['bert_score'] = [r['score'] for r in results]

        # Normalize labels
        label_map = {
            'LABEL_0': 'negative',
            'LABEL_1': 'neutral',
            'LABEL_2': 'positive',
            'NEGATIVE': 'negative',
            'NEUTRAL': 'neutral',
            'POSITIVE': 'positive'
        }
        df['bert_cat'] = df['bert_label'].map(label_map).fillna(df['bert_label'])

        # Compute counts and percentages
        total_reviews = len(df)
        pos = df['bert_cat'].eq('positive').sum()
        neu = df['bert_cat'].eq('neutral').sum()
        neg = df['bert_cat'].eq('negative').sum()

        bert_summary = {
            "outlet_name": outlet_name_clean,
            "total_reviews_bert": total_reviews,
            "positive_bert": int(pos),
            "neutral_bert": int(neu),
            "negative_bert": int(neg),
            "pct_positive_bert": round(pos / total_reviews * 100, 2),
            "pct_neutral_bert": round(neu / total_reviews * 100, 2),
            "pct_negative_bert": round(neg / total_reviews * 100, 2)
        }
        bert_summaries.append(bert_summary)

    except Exception as e:
        print(f"❌ Error processing {outlet_name}: {e}")


# -----------------------------
# Step 5: Merge with VADER summary
# -----------------------------
bert_df = pd.DataFrame(bert_summaries)
if bert_df.empty:
    raise ValueError("❌ No BERT results were generated.")

# Ensure consistent column for merge
vader_df['outlet_clean'] = vader_df['outlet'].str.strip().str.lower()
merged_df = pd.merge(
    vader_df,
    bert_df,
    left_on='outlet_clean',
    right_on='outlet_name',
    how='left'
).drop(columns=['outlet_clean', 'outlet_name'])

# -----------------------------
# Step 6: Save final CSV
# -----------------------------
merged_df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Saved BERT-augmented bottom 20 outlets CSV: {OUTPUT_FILE}")

# Preview
print("\nSample rows:")
print(merged_df.head())

📊 Loaded 20 top 20 outlets from VADER summary
📂 Found 134 outlet CSV files in '/Users/breann/Documents/GitHub/IS434-Anytime-Fitness/Google-Reviews/Reviews/All'
⚙️ Loading BERT model: cardiffnlp/twitter-roberta-base-sentiment ...


Device set to use cpu
Processing outlets: 100%|██████████| 134/134 [05:02<00:00,  2.25s/it]

✅ Saved BERT-augmented bottom 20 outlets CSV: /Users/breann/Documents/GitHub/IS434-Anytime-Fitness/Google-Reviews/Reviews/Overview/top_20_outlets_with_BERT_sentiment.csv

Sample rows:
                                outlet  avg_rating  total_reviews  positive  \
0             Anytime Fitness Bedok 85        4.94           1003       810   
1     Anytime Fitness City Square Mall        4.86            953       858   
2  Anytime Fitness Bukit Timah Central        4.90            882       671   
3          Anytime Fitness Buona Vista        4.86            773       691   
4      Anytime Fitness Havelock Outram        4.91            606       472   

   neutral  negative  no_text  pct_positive  pct_neutral  pct_negative  \
0      184         9      147         80.76        18.34          0.90   
1       75        20       52         90.03         7.87          2.10   
2      189        22      164         76.08        21.43          2.49   
3       62        20       40         89.39  

In [24]:
from pathlib import Path
import pandas as pd

repo_root = Path.cwd().parent
csv_path = (repo_root / "Reviews" / "Overview" / "top_20_outlets_with_BERT_sentiment.csv").resolve()
df = pd.read_csv(csv_path)
df.head()


# df = top_20_with_sentiment.csv already loaded
df['total_vader'] = df['positive'] + df['neutral'] + df['negative']
df['total_bert'] = df['positive_bert'] + df['neutral_bert'] + df['negative_bert']

# Compute agreement in percentages
df['pct_positive_diff'] = abs(df['pct_positive'] - df['pct_positive_bert'])
df['pct_neutral_diff'] = abs(df['pct_neutral'] - df['pct_neutral_bert'])
df['pct_negative_diff'] = abs(df['pct_negative'] - df['pct_negative_bert'])

# Optional: a single measure of disagreement
df['avg_pct_diff'] = df[['pct_positive_diff','pct_neutral_diff','pct_negative_diff']].mean(axis=1)

# Sort by disagreement to see which outlets differ most
df.sort_values('avg_pct_diff', ascending=False)[['outlet','pct_positive','pct_positive_bert','pct_neutral','pct_neutral_bert','pct_negative','pct_negative_bert','avg_pct_diff']]

,outlet,pct_positive,pct_positive_bert,pct_neutral,pct_neutral_bert,pct_negative,pct_negative_bert,avg_pct_diff
11,Anytime Fitness New Upper Thomson,70.41,88.58,28.40,6.69,1.18,4.72,14.473333
5,Anytime Fitness Chai Chee,73.01,92.64,25.00,3.45,1.99,3.91,14.366667
9,Anytime Fitness Jurong Summit,76.47,93.36,21.39,2.99,2.14,3.65,12.266667
2,Anytime Fitness Bukit Timah Central,76.08,93.31,21.43,3.48,2.49,3.20,11.963333
4,Anytime Fitness Havelock Outram,77.89,94.28,20.63,3.35,1.49,2.37,11.516667
0,Anytime Fitness Bedok 85,80.76,95.79,18.34,3.04,0.90,1.17,10.200000
15,Anytime Fitness Yishun East,80.00,93.07,18.75,5.45,1.25,1.49,8.870000
6,Anytime Fitness Ang Mo Kio South,85.90,96.08,12.24,1.65,1.86,2.27,7.060000
19,Anytime Fitness Tekka Place,86.18,96.36,12.20,1.82,1.63,1.82,6.916667
13,Anytime Fitness Sunway Big Box,86.22,94.00,12.01,3.60,1.77,2.40,5.606667
